In [1]:
import os

import json
from openai import OpenAI

import gradio as gr
import ollama
Model = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")


/Users/kutigbolahan/Documents/LLM/LLM-Airline-Assistant/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kutigbolahan/Documents/LLM/LLM-Airline-Assistant/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answers, no more than 1 sentence."
system_message += "Always be accurate. if you don't know the answer, say so"

In [3]:
#The chat function
def chat(message, history):
    messages= [{"role":"system", "content":system_message}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(model=Model, messages=messages)
    return response.choices[0].message.content

In [4]:
gr.ChatInterface(fn=chat, type='messages').launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [5]:
#Tools: we write a function and have the LLM call that function as part of its response
ticket_prices = {"lagos":"40k", "Abuja":"150k", "Benin": "70k", "uyo":"50k", "Abeokuta":"25k"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city,"Unknown")


In [6]:
get_ticket_price("lagos")

Tool get_ticket_price called for lagos


'40k'

In [7]:
#The particular dictionary structure that's required to describe our function:
price_function = {
    "name":"get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters":{"type": "object",
                  "properties":{
        "destination_city":{
            "type":"string",
            "description": "The city that the customer wants to travel to",
        },
    },
     "required":["destination_city"],
                  "additionalProperties":False,
                  },
}

In [8]:
tools = [{"type": "function", "function":price_function}]

In [10]:
#get the llm to call our function
def chat(message, history):
    messages =[{"role":"assistant", "content": system_message}] +history+[{"role":"user", "content":message}]
    response = openai.chat.completions.create(model=Model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=='tool_calls':
        message = response.choices[0].message
        response , city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=Model, messages=messages)
        return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role":"tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id

    }
    return response, city